In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
from scipy.fft import dctn, idctn

In [2]:
def upload_image():
    global img, img_array
    file_path = filedialog.askopenfilename(filetypes=[("BMP files", "*.bmp")])
    if file_path:
        img = Image.open(file_path)
        img_array = np.array(img)
        img.thumbnail((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        img_label.config(image=img_tk)
        img_label.image = img_tk

    div1.pack(padx=10, pady=10, fill="both", expand=True)
    label_F.pack()
    entry_F.pack(pady=10)
    lock_button_F.pack(pady=5)

def lock_F():
    try:
        F = int(entry_F.get())
        if F <= 0:
            raise ValueError
    except ValueError:
        messagebox.showerror("Warning", "F must be a positive integer")
        return

    # Disabilita l'input per F e il pulsante di blocco
    entry_F.config(state='disabled')
    lock_button_F.config(state='disabled')

    # Aggiorna e mostra l'etichetta e l'entry per d
    div2.pack(padx=10, pady=10, fill="both", expand=True)
    # Aggiorna e mostra l'etichetta e l'entry per d
    label_d.config(text=f"Insert value of d (0 ≤ d ≤ {2*F - 2}):")
    label_d.pack()
    entry_d.pack(pady=10)
    lock_button_d.pack(pady=5)


def lock_d():
    try:
        F = int(entry_F.get())
        d = int(entry_d.get())
        if d < 0 or d > (2 * F - 2):
            raise ValueError
    except ValueError:
        messagebox.showerror("Warning", f"d must be an integer between 0 and {2*F - 2}")
        return

    # Disabilita l'input per F e il pulsante di blocco
    entry_d.config(state='disabled')
    lock_button_d.config(state='disabled')

    # Aggiorna e mostra il pulsante per applicare le operazioni DCT
    div3.pack(padx=10, pady=10, fill="both", expand=True)
    apply_button.pack(pady=10)


def apply_dct_operations():
    F = int(entry_F.get())
    d = int(entry_d.get())
    height, width = img_array.shape

    block_height = height // F
    block_width = width // F

    img_dct_array = np.zeros((height, width))
    
    for i in range(block_height):
        for j in range(block_width):
            block_start_i = i * F
            block_end_i = block_start_i + F
            block_start_j = j * F
            block_end_j = block_start_j + F
            block = img_array[block_start_i:block_end_i, block_start_j:block_end_j]
            
            c = dctn(block, norm='ortho')

            for k in range(F):
                for l in range(F):
                    if k + l >= d:
                        c[k, l] = 0

            img_dct_array[block_start_i:block_end_i, block_start_j:block_end_j] = c

            block_idct = idctn(c, norm='ortho')

            # Round and clip block_idct
            block_idct = np.round(block_idct).astype(int)
            block_idct = np.clip(block_idct, 0, 255)

            # Update img_dct_array
            img_dct_array[block_start_i:block_end_i, block_start_j:block_end_j] = block_idct

    modified_img = Image.fromarray(img_dct_array.astype(np.uint8))
    new_window = tk.Toplevel(root)
    new_window.title("Reconstructed Image, F = " + str(F) + ", d = " + str(d))

    modified_img_tk = ImageTk.PhotoImage(modified_img)
    modified_img_label = tk.Label(new_window, image=modified_img_tk)
    modified_img_label.image = modified_img_tk
    modified_img_label.pack()

    resized_original_img = img.resize(modified_img.size, Image.ADAPTIVE)
    # Create a new window for the original image
    original_window = tk.Toplevel(root)
    original_window.title("Original Image")

    original_img_tk = ImageTk.PhotoImage(resized_original_img)
    original_img_label = tk.Label(original_window, image=original_img_tk)
    original_img_label.image = original_img_tk
    original_img_label.pack()
    

def create_div(parent):
    div = tk.Frame(parent, bg="lightgray", bd=2, relief="ridge")
    return div

In [3]:
# Crea la finestra Tkinter
root = tk.Tk()
root.title("DCT2 Image Compression")
root.attributes("-topmost", True)
root.minsize(500, 300)

img = None
img_array = None

# Crea e posiziona i widget
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(pady=10)

img_label = tk.Label(root)
img_label.pack(pady=10)

div1 = create_div(root)

label_F = tk.Label(root, text="Value of F (block dimension):")
entry_F = tk.Entry(root)
lock_button_F = tk.Button(root, text="Confirm", command=lock_F)


div2 = create_div(root)

label_d = tk.Label(root)
entry_d = tk.Entry(root)
lock_button_d = tk.Button(root, text="Confirm", command=lock_d)


div3 = create_div(root)
apply_button = tk.Button(root, text="Run DCT2", command=apply_dct_operations)

# Avvia il loop degli eventi Tkinter
root.mainloop()